# Příkazy použitelné k diplomové práci
Záměrně typo v názvu tabluky v databázi, abych si to omylem nespustil a nerozbil

## Calculate Value for Each Row in Pandas Dataframe
& insert in new column

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import json
import kody
import pandas as pd
import time
from datetime import datetime

analyser = SentimentIntensityAnalyzer() #vypočte Vader sentiment

print(datetime.now().isoformat())

data_frame = pd.read_sql('SELECT time, username, tweet, followers,  sentiment FROM ttTable ORDER BY time ASC', con=kody.cnx) #5s bez order by 8s s order by
data_frame['vader_compound'] = [analyser.polarity_scores(x)['compound'] for x in data_frame['tweet']] #7min

print(data_frame) 

print(datetime.now().isoformat())

## Running Scripts
Kontrola funkčnosti skriptu updatem času v databázi

In [ ]:
import kody
from datetime import datetime
import time

cursor=kody.cnx.cursor()

def is_it_running():
    script_name = "test"
    now = datetime.now().isoformat()
    cursor.execute(
            "INSERT INTO runng_scripts (script, time) VALUES (%s,%s)",
            (script_name, now))
    kody.cnx.commit()
    time.sleep(5)

while True:
    is_it_running()

# Vader
### **! před updatem tabulky zkontrolovat nastavení MySql databáze**
- Update hodnoty sloupce sentiment_vader v sql tabulce pomocí dvou cursorů
- výpočet je hned, ale update tabulky trvá hrozně dlouho, 

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import json
import kody
import pandas as pd
import time
from datetime import datetime
#!!!!!!!!!!!!!!!!!!!!!!!!!!!před spuštěním zkontrolovat nastavení mysql time/published ON UPDATE CURRENT TIMESTAMP při updatu sql změní i hodnotu času!!!!!
cursor=kody.cnx.cursor(buffered=True)
cursor2=kody.cnx.cursor(buffered=True)
analyser = SentimentIntensityAnalyzer()

print(datetime.now().isoformat())

counter = 0

def foo(row):
    global counter
    counter += 1
    print(counter)
    return analyser.polarity_scores(row)["compound"]

cursor=kody.cnx.cursor(buffered=True)
cursor2=kody.cnx.cursor(buffered=True)
analyser = SentimentIntensityAnalyzer()

cursor.execute("SELECT title FROM newsINTC")

cursor2.executemany(
   "UPDATE nesINTC SET sentiment_vader = %s WHERE title = %s LIMIT 1",
   [(foo(row), row[0]) for row in cursor]
)

kody.cnx.commit()

print(datetime.now().isoformat())

# 0HLC Chart
- total duration: 8s

In [ ]:
import kody
import pandas as pd
from datetime import datetime
import plotly.graph_objects as go
import os
print("start: ", datetime.now().isoformat())

# MA bassed on time
def dataframe_resample():
    data_frame = pd.read_sql('SELECT time, sentiment FROM twetTable ORDER BY time ASC', con=kody.cnx)
    data_frame = data_frame.set_index(['time'])
    df_mean = data_frame.resample('30min').mean()
    return df_mean
#print(df_mean)

def ohlc_chart():
    df_mean_OHLC = dataframe_resample()['sentiment'].resample('2h').ohlc()
    fig = go.Figure(data=[go.Candlestick(x=df_mean_OHLC.index,
                    open=df_mean_OHLC['open'], 
                    high=df_mean_OHLC['high'],
                    low=df_mean_OHLC['low'], 
                    close=df_mean_OHLC['close'])
                         ])

    fig.update_layout(xaxis_rangeslider_visible=False)
    fig.write_image("assets/t.png")
    return

ohlc_chart()
print("done: ", datetime.now().isoformat())